# Recommendation with Selenium

## Part 1: Data Acquisition

### 1. Install package - selenium

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


### 2. Import binaries

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### 3. Crawl all reviews

In [3]:
def extract_reviews(url, num_pages):
    driver_path = "C:/Users/jecju/EE4213/Individual Assignment/ chromedriver_win32/chromedriver.exe"
    driver = webdriver.Chrome(executable_path=driver_path)  # Replace with your desired browser driver
    driver.get(url)
    
    reviews = []
    
    for _ in range(num_pages):
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".review"))
        )
        time.sleep(2)
        
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".review")
        
        for review_element in review_elements:
            try:
                rating = review_element.find_element(By.CSS_SELECTOR, ".a-icon-alt").get_attribute("innerHTML").split()[0]
                name = review_element.find_element(By.CSS_SELECTOR, ".a-profile-name").text
                text = review_element.find_element(By.CSS_SELECTOR, ".review-text-content span").text
                date = review_element.find_element(By.CSS_SELECTOR, ".review-date").text
                
                reviews.append({
                    "rating": rating,
                    "name": name,
                    "text": text,
                    "date": date
                })
            except Exception as e:
                print(f"Error: {e}")
        
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, ".a-last a")
            next_button.click()
            time.sleep(2)
        except:
            print("No more pages found.")
            break
    
    driver.quit()
    return reviews

if __name__ == "__main__":
#     url = "https://www.amazon.com/Seagate-Portable-External-Hard-Drive/dp/B07CRG94G3/ref=sr_1_1?crid=1CJWC0CEJ6DES&keywords=Seagate+Portable+2TB+External+Hard+Drive+HDD+%E2%80%94+USB+3.0+for+PC%2C+Mac%2C+PlayStation%2C+%26+Xbox+-1-Year+Rescue+Service+%28STGX2000400%29&qid=1680087978&sprefix=seagate+portable+2tb+external+hard+drive+hdd+usb+3.0+for+pc%2C+mac%2C+playstation%2C+%26+xbox+-1-year+rescue+service+stgx2000400+%2Caps%2C240&sr=8-1"
    url = "https://www.amazon.com/Dell-OptiPlex-Computer-Desktop-Keyboard/dp/B09TN242Q5/ref=sr_1_22?qid=1680104234&s=computers-intl-ship&sr=1-22"
    num_pages = 15
    reviews = extract_reviews(url, num_pages)
    print(reviews)

<ipython-input-3-3312c739d0cb>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)  # Replace with your desired browser driver


[{'rating': '4.0', 'name': 'Norell', 'text': 'So something that wasn’t obvious to me is - the computer dose not support wireless connection to the internet so I’m having to buy a 50 ft ethernet cord so I can connect it to the internet other than not supporting that the computer works and looks fine just need to connect it to my Wi-Fi at home', 'date': 'Reviewed in the United States on March 7, 2023'}, {'rating': '4.0', 'name': 'Anika', 'text': 'It’s a good computer for the price. My son loves it. It was hard because we didn’t get the instructions until after we received and set the computer up. Once I received the email message from the company and followed the directions, it was fine.', 'date': 'Reviewed in the United States on March 6, 2023'}, {'rating': '4.0', 'name': "Ramblin' Guy", 'text': 'Standard home use (checking email, creating documents).\n\nI was disappointed by the lack of an optical drive.', 'date': 'Reviewed in the United States on March 24, 2023'}, {'rating': '3.0', 'n

In [4]:
reviews

[{'rating': '4.0',
  'name': 'Norell',
  'text': 'So something that wasn’t obvious to me is - the computer dose not support wireless connection to the internet so I’m having to buy a 50 ft ethernet cord so I can connect it to the internet other than not supporting that the computer works and looks fine just need to connect it to my Wi-Fi at home',
  'date': 'Reviewed in the United States on March 7, 2023'},
 {'rating': '4.0',
  'name': 'Anika',
  'text': 'It’s a good computer for the price. My son loves it. It was hard because we didn’t get the instructions until after we received and set the computer up. Once I received the email message from the company and followed the directions, it was fine.',
  'date': 'Reviewed in the United States on March 6, 2023'},
 {'rating': '4.0',
  'name': "Ramblin' Guy",
  'text': 'Standard home use (checking email, creating documents).\n\nI was disappointed by the lack of an optical drive.',
  'date': 'Reviewed in the United States on March 24, 2023'},


## Part 2: Data Processing

### 1. Install packages - nltk, pandas

In [5]:
#pip install nltk pandas

### 2. Import Binaries

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jecju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jecju\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jecju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 3. Preprocess data

In [7]:
def clean_review_text(text):
    # Remove special characters, digits, and convert to lowercase
    cleaned_text = ''.join(c for c in text if c.isalpha() or c.isspace()).lower()

    # Tokenize text into words
    words = word_tokenize(cleaned_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # Reconstruct the cleaned, lemmatized text
    return ' '.join(lemmatized_words)

def preprocess_reviews(reviews):
    df = pd.DataFrame(reviews)
    
    # Clean review text
    df['cleaned_text'] = df['text'].apply(clean_review_text)

    return df

if __name__ == "__main__":
    preprocessed_reviews = preprocess_reviews(reviews)
    print(preprocessed_reviews)

    rating             name  \
0      4.0           Norell   
1      4.0            Anika   
2      4.0     Ramblin' Guy   
3      3.0  Saika the Medic   
4      4.0         Steve H.   
..     ...              ...   
115    3.0  Saika the Medic   
116    4.0         Steve H.   
117    4.0  John MAZZarella   
118    4.0            Katie   
119    4.0         Cheslynn   

                                                  text  \
0    So something that wasn’t obvious to me is - th...   
1    It’s a good computer for the price. My son lov...   
2    Standard home use (checking email, creating do...   
3    For the price and everything thing i got this ...   
4    Will not work with a monitor over 1080 resolut...   
..                                                 ...   
115  For the price and everything thing i got this ...   
116  Will not work with a monitor over 1080 resolut...   
117  It took a while to get past the girl at the fr...   
118  Computer work great is the wifi flash chop

In [8]:
preprocessed_reviews

,rating,name,text,date,cleaned_text
0,4.0,Norell,So something that wasn’t obvious to me is - th...,"Reviewed in the United States on March 7, 2023",something wasnt obvious computer dose support ...
1,4.0,Anika,It’s a good computer for the price. My son lov...,"Reviewed in the United States on March 6, 2023",good computer price son love hard didnt get in...
2,4.0,Ramblin' Guy,"Standard home use (checking email, creating do...","Reviewed in the United States on March 24, 2023",standard home use checking email creating docu...
3,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...
4,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...
...,...,...,...,...,...
115,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...
116,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...
117,4.0,John MAZZarella,It took a while to get past the girl at the fr...,"Reviewed in the United States on January 29, 2023",took get past girl front desk help computer se...
118,4.0,Katie,Computer work great is the wifi flash chop tha...,"Reviewed in the United States on February 26, ...",computer work great wifi flash chop need repla...


## Part 3: Data Deployment

### 1. Install package - transformers, torch

In [9]:
pip install transformers torch

### 2. Import binaries

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### 3.1 Sentiment Analysis- cardiffnlp/twitter-roberta-base-sentiment

In [11]:
def sentiment_analysis(preprocessed_reviews):
    # Load the pre-trained sentiment analysis model and tokenizer
    model_name = "cardiffnlp/twitter-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Function to convert sentiment score to sentiment label
    def sentiment_label(score):
        return "positive" if score == 2 else "neutral" if score == 1 else "negative"

    # Perform sentiment analysis on the cleaned text
    sentiments = []
    max_token_length = 128  # Set the maximum token length
    for text in preprocessed_reviews['cleaned_text']:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_token_length)
        outputs = model(**inputs)
        sentiment_score = torch.argmax(outputs.logits, dim=1).item()
        sentiment = sentiment_label(sentiment_score)
        sentiments.append(sentiment)

    preprocessed_reviews['sentiment'] = sentiments
    return preprocessed_reviews

if __name__ == "__main__":
    preprocessed_reviews = preprocess_reviews(reviews)
    sentiment_reviews1 = sentiment_analysis(preprocessed_reviews)
    print(sentiment_reviews1)

    rating             name  \
0      4.0           Norell   
1      4.0            Anika   
2      4.0     Ramblin' Guy   
3      3.0  Saika the Medic   
4      4.0         Steve H.   
..     ...              ...   
115    3.0  Saika the Medic   
116    4.0         Steve H.   
117    4.0  John MAZZarella   
118    4.0            Katie   
119    4.0         Cheslynn   

                                                  text  \
0    So something that wasn’t obvious to me is - th...   
1    It’s a good computer for the price. My son lov...   
2    Standard home use (checking email, creating do...   
3    For the price and everything thing i got this ...   
4    Will not work with a monitor over 1080 resolut...   
..                                                 ...   
115  For the price and everything thing i got this ...   
116  Will not work with a monitor over 1080 resolut...   
117  It took a while to get past the girl at the fr...   
118  Computer work great is the wifi flash chop

In [12]:
sentiment_reviews1

,rating,name,text,date,cleaned_text,sentiment
0,4.0,Norell,So something that wasn’t obvious to me is - th...,"Reviewed in the United States on March 7, 2023",something wasnt obvious computer dose support ...,positive
1,4.0,Anika,It’s a good computer for the price. My son lov...,"Reviewed in the United States on March 6, 2023",good computer price son love hard didnt get in...,positive
2,4.0,Ramblin' Guy,"Standard home use (checking email, creating do...","Reviewed in the United States on March 24, 2023",standard home use checking email creating docu...,negative
3,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...,positive
4,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...,positive
...,...,...,...,...,...,...
115,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...,positive
116,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...,positive
117,4.0,John MAZZarella,It took a while to get past the girl at the fr...,"Reviewed in the United States on January 29, 2023",took get past girl front desk help computer se...,positive
118,4.0,Katie,Computer work great is the wifi flash chop tha...,"Reviewed in the United States on February 26, ...",computer work great wifi flash chop need repla...,positive


### 3.2 Analysis

In [13]:
import pandas as pd

def analyze_sentiment_reviews(sentiment_reviews):
    # Convert the 'rating' column to a numeric data type
    sentiment_reviews['rating'] = pd.to_numeric(sentiment_reviews['rating'])

    # Calculate the average rating
    avg_rating = sentiment_reviews['rating'].mean()
    print(f"Average rating: {avg_rating:.2f}")

    # Calculate the number of reviews for each sentiment
    sentiment_counts = sentiment_reviews['sentiment'].value_counts()
    print("\nNumber of reviews by sentiment:")
    print(sentiment_counts)

    # Calculate the percentage of reviews for each sentiment
    sentiment_percents = sentiment_reviews['sentiment'].value_counts(normalize=True) * 100
    print("\nPercentage of reviews by sentiment:")
    print(sentiment_percents)

    # Calculate the average rating for each sentiment
    avg_rating_by_sentiment = sentiment_reviews.groupby('sentiment')['rating'].mean()
    print("\nAverage rating by sentiment:")
    print(avg_rating_by_sentiment)

    return {
        "average_rating": avg_rating,
        "sentiment_counts": sentiment_counts,
        "sentiment_percents": sentiment_percents,
        "avg_rating_by_sentiment": avg_rating_by_sentiment,
    }

if __name__ == "__main__":
    analysis_results = analyze_sentiment_reviews(sentiment_reviews1)

Average rating: 3.88

Number of reviews by sentiment:
positive    105
negative     15
Name: sentiment, dtype: int64

Percentage of reviews by sentiment:
positive    87.5
negative    12.5
Name: sentiment, dtype: float64

Average rating by sentiment:
sentiment
negative    4.000000
positive    3.857143
Name: rating, dtype: float64


### 4.1 Sentiment Analysis - nlptown/bert-base-multilingual-uncased-sentiment

In [14]:
def sentiment_analysis(preprocessed_reviews):
    # Load the pre-trained sentiment analysis model and tokenizer
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Function to convert sentiment score to sentiment label
    def sentiment_label(score):
        return "positive" if score >= 4 else "neutral" if score == 3 else "negative"

    # Perform sentiment analysis on the cleaned text
    sentiments = []
    max_token_length = 512  # Set the maximum token length
    for text in preprocessed_reviews['cleaned_text']:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_token_length)
        outputs = model(**inputs)
        sentiment_score = torch.argmax(outputs.logits, dim=1).item() + 1  # The model's output is 0-based
        sentiment = sentiment_label(sentiment_score)
        sentiments.append(sentiment)

    preprocessed_reviews['sentiment'] = sentiments
    return preprocessed_reviews

if __name__ == "__main__":
    preprocessed_reviews = preprocess_reviews(reviews)
    sentiment_reviews2 = sentiment_analysis(preprocessed_reviews)
    print(sentiment_reviews2)

    rating             name  \
0      4.0           Norell   
1      4.0            Anika   
2      4.0     Ramblin' Guy   
3      3.0  Saika the Medic   
4      4.0         Steve H.   
..     ...              ...   
115    3.0  Saika the Medic   
116    4.0         Steve H.   
117    4.0  John MAZZarella   
118    4.0            Katie   
119    4.0         Cheslynn   

                                                  text  \
0    So something that wasn’t obvious to me is - th...   
1    It’s a good computer for the price. My son lov...   
2    Standard home use (checking email, creating do...   
3    For the price and everything thing i got this ...   
4    Will not work with a monitor over 1080 resolut...   
..                                                 ...   
115  For the price and everything thing i got this ...   
116  Will not work with a monitor over 1080 resolut...   
117  It took a while to get past the girl at the fr...   
118  Computer work great is the wifi flash chop

In [15]:
sentiment_reviews2

,rating,name,text,date,cleaned_text,sentiment
0,4.0,Norell,So something that wasn’t obvious to me is - th...,"Reviewed in the United States on March 7, 2023",something wasnt obvious computer dose support ...,neutral
1,4.0,Anika,It’s a good computer for the price. My son lov...,"Reviewed in the United States on March 6, 2023",good computer price son love hard didnt get in...,positive
2,4.0,Ramblin' Guy,"Standard home use (checking email, creating do...","Reviewed in the United States on March 24, 2023",standard home use checking email creating docu...,negative
3,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...,negative
4,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...,neutral
...,...,...,...,...,...,...
115,3.0,Saika the Medic,For the price and everything thing i got this ...,"Reviewed in the United States on October 1, 2022",price everything thing got great computer mont...,negative
116,4.0,Steve H.,Will not work with a monitor over 1080 resolut...,"Reviewed in the United States on January 15, 2023",work monitor resolution seems fast im gamer ne...,neutral
117,4.0,John MAZZarella,It took a while to get past the girl at the fr...,"Reviewed in the United States on January 29, 2023",took get past girl front desk help computer se...,neutral
118,4.0,Katie,Computer work great is the wifi flash chop tha...,"Reviewed in the United States on February 26, ...",computer work great wifi flash chop need repla...,positive


### 4.2 Anaylsis

In [16]:
import pandas as pd

def analyze_sentiment_reviews(sentiment_reviews):
    # Convert the 'rating' column to a numeric data type
    sentiment_reviews['rating'] = pd.to_numeric(sentiment_reviews['rating'])

    # Calculate the average rating
    avg_rating = sentiment_reviews['rating'].mean()
    print(f"Average rating: {avg_rating:.2f}")

    # Calculate the number of reviews for each sentiment
    sentiment_counts = sentiment_reviews['sentiment'].value_counts()
    print("\nNumber of reviews by sentiment:")
    print(sentiment_counts)

    # Calculate the percentage of reviews for each sentiment
    sentiment_percents = sentiment_reviews['sentiment'].value_counts(normalize=True) * 100
    print("\nPercentage of reviews by sentiment:")
    print(sentiment_percents)

    # Calculate the average rating for each sentiment
    avg_rating_by_sentiment = sentiment_reviews.groupby('sentiment')['rating'].mean()
    print("\nAverage rating by sentiment:")
    print(avg_rating_by_sentiment)

    return {
        "average_rating": avg_rating,
        "sentiment_counts": sentiment_counts,
        "sentiment_percents": sentiment_percents,
        "avg_rating_by_sentiment": avg_rating_by_sentiment,
    }

if __name__ == "__main__":
    analysis_results = analyze_sentiment_reviews(sentiment_reviews2)

Average rating: 3.88

Number of reviews by sentiment:
neutral     45
positive    45
negative    30
Name: sentiment, dtype: int64

Percentage of reviews by sentiment:
neutral     37.5
positive    37.5
negative    25.0
Name: sentiment, dtype: float64

Average rating by sentiment:
sentiment
negative    3.5
neutral     4.0
positive    4.0
Name: rating, dtype: float64


## Part 4: Recommendation

### 1. Install packages

In [17]:
#pip install gensim --user

### 2. Import binaries

In [18]:
from gensim import corpora
from gensim.models import LdaModel

C:\Users\jecju\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### 3. Extract Topics from Reviews

In [19]:
def extract_topics(sentiment_reviews, num_topics=5, num_words=5):
    # Tokenize the cleaned text and remove stopwords
    stop_words = set(stopwords.words("english"))
    tokenized_reviews = [
        [word for word in word_tokenize(text) if word not in stop_words]
        for text in sentiment_reviews['cleaned_text']
    ]

    # Create a dictionary and a corpus from the tokenized reviews
    dictionary = corpora.Dictionary(tokenized_reviews)
    corpus = [dictionary.doc2bow(review) for review in tokenized_reviews]

    # Build an LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Extract topics with their top words
    topics = lda_model.print_topics(num_words=num_words)

    return topics

In [24]:
if __name__ == "__main__":
    sentiment_reviews = sentiment_reviews2
    topics = extract_topics(sentiment_reviews, num_topics=5, num_words=5)
    for idx, topic in topics:
        print(f"Topic {idx + 1}: {topic}")

Topic 1: 0.061*"computer" + 0.061*"received" + 0.061*"email" + 0.031*"love" + 0.031*"get"
Topic 2: 0.061*"computer" + 0.061*"screw" + 0.031*"need" + 0.031*"work" + 0.031*"great"
Topic 3: 0.031*"computer" + 0.031*"internet" + 0.031*"work" + 0.031*"connect" + 0.031*"support"
Topic 4: 0.044*"love" + 0.029*"speaker" + 0.029*"side" + 0.029*"light" + 0.029*"tower"
Topic 5: 0.042*"computer" + 0.035*"drive" + 0.035*"hard" + 0.021*"need" + 0.021*"month"


### 4. Provide Recommendation

In [25]:
def sentiment_statistics(sentiment_reviews):
    total_reviews = len(sentiment_reviews)
    sentiment_counts = sentiment_reviews['sentiment'].value_counts()
    sentiment_percentages = sentiment_counts / total_reviews * 100

    statistics = pd.DataFrame({
        'count': sentiment_counts,
        'percentage': sentiment_percentages
    })

    return statistics

In [28]:
def generate_recommendations(sentiment_reviews, topics):
    sentiment_stats = sentiment_statistics(sentiment_reviews)
    print("Sentiment Statistics:")
    print(sentiment_stats)

    print("\nTopics:")
    for idx, topic in topics:
        print(f"Topic {idx + 1}: {topic}")

    print("\nRecommendations:")

    # Positive feedback
    if sentiment_stats.loc["positive", "percentage"] > 75:
        print("- Keep up the good work! The majority of customers have positive feedback.")
    else:
        print("- Strive to improve the overall customer experience as the positive feedback percentage is below 75%.")

    # Negative feedback
    if sentiment_stats.loc["negative", "percentage"] > 25:
        print("- Pay attention to the issues mentioned in the negative reviews and address them to improve customer satisfaction.")
    else:
        print("- Keep monitoring negative feedback to maintain a low percentage of negative reviews.")

    # Topic-based recommendations
    for idx, topic in topics:
        print(f"\nFor Topic {idx + 1}:")
        print(f"- Focus on the keywords: {topic}")

In [29]:
if __name__ == "__main__":
    topics = extract_topics(sentiment_reviews, num_topics=5, num_words=5)
    generate_recommendations(sentiment_reviews, topics)

Sentiment Statistics:
          count  percentage
neutral      45        37.5
positive     45        37.5
negative     30        25.0

Topics:
Topic 1: 0.085*"computer" + 0.043*"fine" + 0.043*"internet" + 0.043*"received" + 0.043*"connect"
Topic 2: 0.040*"monitor" + 0.040*"resolution" + 0.020*"use" + 0.020*"work" + 0.020*"seems"
Topic 3: 0.043*"computer" + 0.031*"drive" + 0.031*"hard" + 0.018*"need" + 0.018*"month"
Topic 4: 0.005*"computer" + 0.005*"hard" + 0.005*"new" + 0.005*"need" + 0.005*"drive"
Topic 5: 0.038*"love" + 0.025*"work" + 0.025*"speaker" + 0.025*"side" + 0.025*"light"

Recommendations:
- Strive to improve the overall customer experience as the positive feedback percentage is below 75%.
- Keep monitoring negative feedback to maintain a low percentage of negative reviews.

For Topic 1:
- Focus on the keywords: 0.085*"computer" + 0.043*"fine" + 0.043*"internet" + 0.043*"received" + 0.043*"connect"

For Topic 2:
- Focus on the keywords: 0.040*"monitor" + 0.040*"resolution" 